In [1]:
!gdown --id 10Yv6xUd1ufDilPcLdhR8L-zQZX5yNwSZ

Downloading...
From: https://drive.google.com/uc?id=10Yv6xUd1ufDilPcLdhR8L-zQZX5yNwSZ
To: /content/dem-vs-rep.zip
9.29MB [00:00, 146MB/s]


In [2]:
import os
import zipfile

local_zip = './dem-vs-rep.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
train_tweets = os.path.join('./dem-vs-rep/train.csv')
test_tweets = os.path.join('./dem-vs-rep/test.csv')

In [4]:
import pandas
train_df = pandas.read_csv(train_tweets)
test_df = pandas.read_csv(test_tweets)
print(train_df[:1])
print(test_df[:1])

      Party         Handle                                              Tweet
0  Democrat  RepDarrenSoto  Today, Senate Dems vote to #SaveTheInternet. P...
      Party        Handle                                              Tweet
0  Democrat  RepAdamSmith  Today the House passed an omnibus spending bil...


In [5]:
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [9]:
import numpy as np

train_tweets = []
train_labels = []

for i in range(len(train_df.index)):
  pre_tweet = train_df.iloc[i]['Tweet']
  post_tweet = ""
  for word in pre_tweet.split():
    if word not in stopwords:
      post_tweet += word + " "
  train_tweets.append(post_tweet)
  party = (train_df.iloc[i]['Party'])
  if party == 'Democrat':
    train_labels.append(0)
  else:
    train_labels.append(1)

test_tweets = []
test_labels = []

for i in range(len(test_df.index)):
  pre_tweet = train_df.iloc[i]['Tweet']
  post_tweet = ""
  for word in pre_tweet.split():
    if word not in stopwords:
      post_tweet += word + " "
  test_tweets.append(post_tweet)
  party = (test_df.iloc[i]['Party'])
  if party == 'Democrat':
    test_labels.append(0)
  else:
    test_labels.append(1)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [10]:
print(len(train_tweets))

69107


In [11]:
print(train_tweets[:2])
print(train_labels[:2])

print(test_tweets[:2])
print(test_labels[:2])

['Today, Senate Dems vote #SaveTheInternet. Proud support similar #NetNeutrality legislation House… https://t.co/n3tggDLU1L ', 'RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher one several recognized @RepDarrenSoto National Teacher Apprecia… ']
[0 0]
['Today, Senate Dems vote #SaveTheInternet. Proud support similar #NetNeutrality legislation House… https://t.co/n3tggDLU1L ', 'RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher one several recognized @RepDarrenSoto National Teacher Apprecia… ']
[0 0]


In [19]:
vocab_size = 69701
embedding_dim = 16
max_length = 
padding_type='post'
trunc_type='post'
OOV_token = "<OOV>"

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=OOV_token)
tokenizer.fit_on_texts(train_tweets)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_tweets)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_tweets)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [22]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[0]))
print(train_tweets[0])

today senate dems vote savetheinternet proud support similar netneutrality legislation house… https t co n3tggdlu1l ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Today, Senate Dems vote #SaveTheInternet. Proud support similar #NetNeutrality legislation House… https://t.co/n3tggDLU1L 


In [23]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           1115216   
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 1,126,749
Trainable params: 1,126,749
Non-trainable params: 0
_________________________________________________________________


In [24]:
num_epochs = 20
model.fit(train_padded, train_labels, epochs=num_epochs, 
          validation_data=(testing_padded, test_labels))

Epoch 1/20
2160/2160 [==============================] - 31s 14ms/step - loss: 0.2384 - accuracy: 0.9036 - val_loss: 0.3190 - val_accuracy: 0.9458
Epoch 2/20
2160/2160 [==============================] - 32s 15ms/step - loss: 0.0940 - accuracy: 0.9720 - val_loss: 0.4104 - val_accuracy: 0.9499
Epoch 3/20
2160/2160 [==============================] - 31s 15ms/step - loss: 0.0494 - accuracy: 0.9844 - val_loss: 0.5230 - val_accuracy: 0.9512
Epoch 4/20
2160/2160 [==============================] - 32s 15ms/step - loss: 0.0257 - accuracy: 0.9929 - val_loss: 0.6899 - val_accuracy: 0.9527
Epoch 5/20
2160/2160 [==============================] - 32s 15ms/step - loss: 0.0146 - accuracy: 0.9961 - val_loss: 0.6611 - val_accuracy: 0.9532
Epoch 6/20
2160/2160 [==============================] - 32s 15ms/step - loss: 0.0090 - accuracy: 0.9981 - val_loss: 0.7897 - val_accuracy: 0.9535
Epoch 7/20
2160/2160 [==============================] - 31s 14ms/step - loss: 0.0062 - accuracy: 0.9988 - val_loss: 0.8779 -

In [59]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(69701, 16)


In [67]:
sentence = ["Freddy Starr ate my hamster"]
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[1, 1, 1, 42, 1]]


In [68]:
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[0.9840058]]
